<a href="https://colab.research.google.com/github/bhaduridba/beam/blob/main/Ikea_assignment_1_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install apache_beam

In [ ]:
pip install datetime

In [ ]:
import apache_beam as beam
p = beam.Pipeline()

#def format_result(sum_pair):
#    customer_name, amount = sum_pair
#    return customer_name + ',' + str(amount) + ' total amount purchased'

def calculate_distinct_age_brackets(element):
    customer_tuple = element.split(';')
    age_bracket = str(customer_tuple[3])

    return age_bracket

calculate_distinct_age_brackets = (
                 p
                 |'read data' >> beam.io.ReadFromText('sample_data/RetailCustomerSales.csv', skip_header_lines=1)
                 |'calculate customer age brackets' >> beam.Map(calculate_distinct_age_brackets)
                 |'Deduplicate elements' >> beam.Distinct()
                 |'write output data' >> beam.io.WriteToText('output4/distinct_age_brackets')
                 )


p.run()

In [ ]:
!cat output4/distinct_age_brackets-00000-of-00001

0-17
55+
26-35
46-50
51-55
36-45
18-25


In [ ]:
!cp output4/distinct_age_brackets-00000-of-00001 sample_data/distinct_age_brackets.txt

In [ ]:
!cat sample_data/distinct_age_brackets.txt

0-17
55+
26-35
46-50
51-55
36-45
18-25


In [ ]:
import apache_beam as beam
p = beam.Pipeline()

def calculate_customer_age_bracket_rating(element):
    customer_age_bracket_rating_tuple = element.split(';')
    customer_name = customer_age_bracket_rating_tuple[0]
    customer_age_bracket = str(customer_age_bracket_rating_tuple[3])
    customer_age_bracket_lowest = customer_age_bracket[0][0]
    customer_age_bracket_lower = customer_age_bracket[0:2]
    customer_age_bracket_higher = customer_age_bracket[3:2]
    customer_age_bracket_highest =  customer_age_bracket[2]

    customer_age_point = 0
    customer_age_class = ''

    #customer _age_scale 6
    if  customer_age_bracket == '55+':
        customer_age_point = 6
        customer_age_class = 'Senior'

    #customer _age_scale 5
    if  customer_age_bracket == '51-55':
        customer_age_point = 5
        customer_age_class = 'Old'

    #customer _age_scale 4
    if  customer_age_bracket == '46-50':
        customer_age_point = 4
        customer_age_class = 'Old Adult'

    #customer _age_scale 3
    if  customer_age_bracket == '36-45':
        customer_age_point = 3
        customer_age_class = 'Middle Age'

    #customer _age_scale 2
    if  customer_age_bracket == '26-35':
        customer_age_point = 2
        customer_age_class = 'Adult'

    #customer _age_scale 1
    if  customer_age_bracket == '18-25':
        customer_age_point = 1
        customer_age_class = 'Young Adult'

    #customer _age_scale 0
    if  customer_age_bracket == '0-17':
        customer_age_point = 0
        customer_age_class = 'Young'

    return customer_name, customer_age_bracket, customer_age_point, customer_age_class

calculate_age_brackets_grading = (
                 p
                 |'read data' >> beam.io.ReadFromText('sample_data/RetailCustomerSales.csv', skip_header_lines=1)
                 |'calculate customer age_brackets' >> beam.Map(calculate_customer_age_bracket_rating)
                 |'write output data' >> beam.io.WriteToText('output9/customer_age_bracket_rating')
                 )

p.run()

In [ ]:
!cat output9/customer_age_bracket_rating-00000-of-00001 | head -20

('1000001', '0-17', 0, 'Young')
('1000001', '0-17', 0, 'Young')
('1000001', '0-17', 0, 'Young')
('1000001', '0-17', 0, 'Young')
('1000002', '55+', 6, 'Senior')
('1000003', '26-35', 2, 'Adult')
('1000004', '46-50', 4, 'Old Adult')
('1000004', '46-50', 4, 'Old Adult')
('1000004', '46-50', 4, 'Old Adult')
('1000005', '26-35', 2, 'Adult')
('1000005', '26-35', 2, 'Adult')
('1000005', '26-35', 2, 'Adult')
('1000005', '26-35', 2, 'Adult')
('1000005', '26-35', 2, 'Adult')
('1000006', '51-55', 5, 'Old')
('1000006', '51-55', 5, 'Old')
('1000006', '51-55', 5, 'Old')
('1000006', '51-55', 5, 'Old')
('1000007', '36-45', 3, 'Middle Age')
('1000008', '26-35', 2, 'Adult')


## Step 1: We will create a Python Data Generator that will stream data from Online Event streams into a Pub Sub topic

## Step 2: We will create a Beam Pipeline as below that will get data from Pub Sub topic and do the processing

# The processing will check for certain Customers IDs and new Customers purchase amount and we can create some sort of Campaign to target based on those metrics 

In [ ]:
import apache_beam as beam
p = beam.Pipeline()

#def format_result(sum_pair):
#    customer_name, amount = sum_pair
#    return customer_name + ',' + str(amount) + ' total amount purchased'

def get_customerIDs(element):
    customer_tuple = element.split(';')
    customerIDs = str(customer_tuple[0])

    return customerIDs

get_customerIDs = (
                 p
                 |'read data' >> beam.io.ReadFromText('sample_data/RetailCustomerSales.csv', skip_header_lines=1)
                 |'get customerIDs' >> beam.Map(get_customerIDs)
                 |'Deduplicate elements' >> beam.Distinct()
                 |'write output data' >> beam.io.WriteToText('output11/customerIDs')
                 )


p.run()

In [ ]:
!cp output11/customerIDs-00000-of-00001 sample_data/customerIDs.txt

In [ ]:
side_list=list()
with open ('sample_data/customerIDs.txt','r') as my_file:
  for customerID in my_file:
    side_list.append(customerID.rstrip())


In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
import os
from apache_beam import window
from apache_beam.transforms.trigger import AfterWatermark, AfterProcessingTime, AccumulationMode, AfterCount

# Replace with your service account path
service_account_path = ''

print("Service account file : ", service_account_path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_path

# Replace with your input subscription id
input_subscription = ''

# OPTION 1: Replace with your output subscription id
output_topic = ''
# OPTION 2: We can also write into GCS bucket storage or in CLoud SQL as per Use case

options = PipelineOptions()
options.view_as(StandardOptions).streaming = True

p = beam.Pipeline(options=options)

def encode_byte_string(element):

   element = str(element)
   return element.encode('utf-8')

pubsub_retaildata= (
                p 
                | 'Read from pub sub' >> beam.io.ReadFromPubSub(subscription= input_subscription,timestamp_attribute = 1553578219)  
                # Customer name,Malmö,Item,Item Category1,200,1553578219/r/n
                | 'Remove extra chars' >> beam.Map(lambda retaildata: (retaildata.rstrip().lstrip()))          # Customer name,Malmö,Item,Item Category1,200,1553578219
                | 'Split Row' >> beam.Map(lambda row : row.split(';'))                             # [Customer name;Malmö;Item;Item Category1;200;1553578219]
                | 'Form Key Value pair' >> beam.Map(lambda elements : (elements[0], float(elements[11])))  # Customer name 112,0
                | 'Window' >> beam.WindowInto(window.FixedWindows(20))
                | 'Sum values' >> beam.CombinePerKey(sum)
                | 'Encode to byte string' >> beam.Map(encode_byte_string)  #Pubsub takes data in form of byte strings 
                #| 'Write to pus sub' >> beam.io.WriteToPubSub(output_topic) # or we can write to GCS bucket
                |'write output data' >> beam.io.WriteToText('output12/customer_and_amount_purchased.csv')
                 
	             )

result = p.run()
result.wait_until_finish()

In [ ]:
p = beam.Pipeline()

# We can pass side inputs to a ParDo transform, which will get passed to its process method.
# we can use Lower bound and Upper bound as mock up for Customer IDs we wish to check activities

class FilterUsingCustomerIDs(beam.DoFn):
  def process(self, element,side_list,lower_bound, upper_bound=float('inf')):
    id = element.split(';')[0]
    id=id.decode('utf-8','ignore').encode("utf-8")
    element_list= element.split(';')
    if (lower_bound <= id <= upper_bound) or id not in side_list:
      return [element_list]

# using pardo to filter Customers with activities 
Check_Customers_activities =( 
                p
                | "Read from text file" >> beam.io.ReadFromText('output12/customer_and_amount_purchased.csv')
                | "ParDo with side inputs" >> beam.ParDo(FilterUsingCustomerIDs(),side_list,100000,200000) 
                | beam.Map(lambda record: (record[0]+ " " + record[1], 1))
                | beam.CombinePerKey(sum)
                | 'Write results' >> beam.io.WriteToText('data/output_new_final')
             )

p.run()